In [1]:
import torch

class ConvModel(torch.nn.Module):
    def __init__(self, channels: int, filter_length: int = 3):
        """
        channels: Number of input/output channels.
        seq_len: (Optional) Expected sequence length (not strictly required for the model definition).
        """
        super().__init__()
        self.conv = torch.nn.Conv1d(
            in_channels=channels,
            out_channels=channels,
            kernel_size=filter_length,
            groups=channels,
            bias=False,
        )
        self.left_padding = filter_length - 1
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.nn.functional.pad(x, (self.left_padding, 0))
        return self.conv(x)

In [2]:
channels = 32
filter_length = 3
x = torch.randn(1, channels, 128)
conv = ConvModel(channels, filter_length)
x = conv(x)
x.shape

torch.Size([1, 32, 128])

In [3]:
from torch.export import export, ExportedProgram
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner
from executorch.exir import EdgeProgramManager, ExecutorchProgramManager, to_edge_transform_and_lower
from executorch.exir.backend.backend_api import to_backend

SEQ_LEN = 256
FILTER_LENGTH = 3
CHANNELS = 4
model = ConvModel(channels=CHANNELS, filter_length=FILTER_LENGTH)
example_input = torch.randn(1, CHANNELS, SEQ_LEN)

exported_program: ExportedProgram = export(model, (example_input,))
edge_program: EdgeProgramManager = to_edge_transform_and_lower(exported_program, partitioner=[XnnpackPartitioner()])
print(edge_program.exported_program().graph_module)

GraphModule(
  (lowered_module_0): LoweredBackendModule()
)



def forward(self, x):
    lowered_module_0 = self.lowered_module_0
    executorch_call_delegate = torch.ops.higher_order.executorch_call_delegate(lowered_module_0, x);  lowered_module_0 = x = None
    getitem = executorch_call_delegate[0];  executorch_call_delegate = None
    return (getitem,)
    
# To see more debug info, please use `graph_module.print_readable()`


In [4]:
from executorch.devtools.backend_debug import get_delegation_info
from tabulate import tabulate
graph_module = edge_program.exported_program().graph_module
delegation_info = get_delegation_info(graph_module)
print(delegation_info.get_summary())
df = delegation_info.get_operator_delegation_dataframe()
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

Total delegated subgraphs: 1
Number of delegated nodes: 2
Number of non-delegated nodes: 1

╒════╤══════════════════════════════╤═══════════════════════════════════╤═══════════════════════════════════════╕
│    │ op_type                      │   occurrences_in_delegated_graphs │   occurrences_in_non_delegated_graphs │
╞════╪══════════════════════════════╪═══════════════════════════════════╪═══════════════════════════════════════╡
│  0 │ aten_constant_pad_nd_default │                                 1 │                                     0 │
├────┼──────────────────────────────┼───────────────────────────────────┼───────────────────────────────────────┤
│  1 │ aten_convolution_default     │                                 1 │                                     0 │
├────┼──────────────────────────────┼───────────────────────────────────┼───────────────────────────────────────┤
│  2 │ getitem                      │                                 0 │                                     